In [1]:
from __future__ import print_function
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)
import versor as vsr
from versor.drawing import *
from motor_estimation import MotorEstimationSolver
from game import VDMotorEstimationSolver

Populating the interactive namespace from numpy and matplotlib


# Generate motors

In [50]:
n_motors = 10
motors = [((vsr.Vec(*np.random.random(3)).unit() * np.random.uniform(-100,100)).trs() * 
           vsr.Rot(vsr.Biv(*np.random.random(3)).unit() * np.random.uniform(-pi,pi))) for i in range(n_motors)]

# Generate lines

In [51]:
n_lines = 10
lines_a = [vsr.Dll(vsr.Vec(*np.random.random(3)).null(), 
                   vsr.Vec(*np.random.random(3)).unit()).unit() for i in range(n_lines)]
lines_b = [[line.spin(motor) for line in lines_a] for motor in motors]
print(len(lines_b))

10


In [54]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(lines_a, lines_b[0]):
    mes.add_line_correspondences_residual_block(a,b)
#     mes.add_line_commutator_residual_block(a,b)
#     mes.add_line_dual_angle_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = 'DENSE_QR'
mes.num_threads = 100
mes.num_linear_solver_threads = 100
(estimated_motor, summary, _) = mes.solve()
print(summary['full_report'])
print(estimated_motor)


Solver Summary (v 1.12.0-eigen-(3.3.3)-lapack-suitesparse-(4.5.4)-cxsparse-(3.1.9)-openmp)

                                     Original                  Reduced
Parameter blocks                            1                        1
Parameters                                  8                        8
Effective parameters                        6                        6
Residual blocks                            10                       10
Residual                                   60                       60

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given                     Used
Linear solver                        DENSE_QR                 DENSE_QR
Threads                                   100                      100
Linear solver threads                     100                      100
Linear solver ordering              AUTOMATIC           

In [34]:
motors[0]

Mot: [ 0.99 -0.029 -0.085 -0.089 -8.4 -7.3 -10 0.42 ]

In [35]:
def daniilidis_motor(LAs, LBs):
    Ds = []
    for LA, LB in zip(LAs, LBs):
        D = np.zeros((8,8))
        for i in range(8):
            ei = vsr.Mot(0,0,0,0,0,0,0,0)
            ei[i] = 1.0
            D[:,i] = np.array(ei * LA - LB * ei)
        Ds.append(D[1:7,:].copy())
    
    Ds = np.array(Ds).reshape(-1,8)
    [U, s, Vt] = np.linalg.svd(Ds)

    v7 = Vt.T[:,-2].copy()
    v8 = Vt.T[:,-1].copy()
    
    v7 = np.array([v7[0], v7[3], -v7[2], v7[1], -v7[7],v7[4], v7[5], v7[6]])
    v8 = np.array([v8[0], v8[3], -v8[2], v8[1], -v8[7],v8[4], v8[5], v8[6]])
    
    u1 = v7[:4]
    v1 = v7[4:]
    u2 = v8[:4]
    v2 = v8[4:]

    a = np.inner(u1,v1)
    b = np.inner(u1,v2) + np.inner(u2,v1)
    c = np.inner(u2,v2)
    [s1, s2] = np.roots([a,b,c])

    val1 = (s1**2 * np.inner(u1,u1)) + (2 * s1 * np.inner(u1,u2)) + (np.inner(u2,u2))
    val2 = (s2**2 * np.inner(u1,u1)) + (2 * s2 * np.inner(u1,u2)) + (np.inner(u2,u2))
    
    if val1 > val2:
        s = s1
        val = val1
    else:
        s = s2
        val = val2

    lambda2 = np.sqrt(1./val)
    lambda1 = s * lambda2
    
    m_arr = lambda1 * Vt.T[:,-2] + lambda2 * Vt.T[:,-1]

    return vsr.Mot(*m_arr)

In [36]:
daniilidis_motor(lines_a, lines_b[0])

Mot: [ -0.99 0.029 0.085 0.089 8.4 7.3 10 -0.42 ]